In [1]:
import torch as tc
import numpy as np
import pickle
import os
from wish_utils import *
from WishDataSet import WishTubeFrameDataSet, get_transform, WishWorkspaceDataSet
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import utils
import threading
import multiprocessing as mp
from tqdm.notebook import tqdm

In [2]:
model_path = '/mnt/ceph/home/wj1132075/Desktop/CNN_Model_Data/TbyT_rot_ax0ax1_withneighb_fasterrcnn_resnet50_fpn.pt'

In [3]:
device = tc.device('cuda') if tc.cuda.is_available() else tc.device('cpu')
device

device(type='cuda')

In [4]:
ws_name="WISH00044720"
workspace_path = f"/mnt/ceph/home/wj1132075/Desktop/CNN_Model_Data/Inferences/RealData/{ws_name}.npz"

In [5]:
ds = WishWorkspaceDataSet(workspace_path)

data set create for /mnt/ceph/home/wj1132075/Desktop/CNN_Model_Data/Inferences/RealData/WISH00044720.npz shape(1520, 128, 1540)


In [6]:
len(ds)

1520

In [7]:
x = ds[0]
x.shape, type(x), x.device

(torch.Size([3, 128, 1540]), torch.Tensor, device(type='cpu'))

In [8]:
data_loader = tc.utils.data.DataLoader(ds, batch_size=8, shuffle=False, num_workers=4)

In [9]:
len(data_loader)

190

In [10]:
tc.cuda.is_available() , tc.cuda.device_count()

(True, 1)

In [11]:
model = get_object_detection_model(num_classes=2)
model.load_state_dict(tc.load(model_path, map_location=device))
model = model.to(device)

/home/wj1132075/.conda/envs/cuda_1/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/wj1132075/.conda/envs/cuda_1/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [12]:
# ws = np.load(workspace_path)['arr_0']

In [13]:
# ws.shape

In [14]:
def plot_tube_detections(tube_idx, bin_data, predicted_boxes, score_threshold = 0.75):
    fig = plt.figure(figsize=(6,6))
    ax = fig.add_subplot(111)
    cs = plt.contourf(bin_data)
    plt.colorbar(cs)
    
    plotted_predictions = 0
    for box, score in zip(predicted_boxes['boxes'], predicted_boxes['scores']):
        # print(f"prediction={box} score={score}")
        x_hat, y_hat = (box[0]+box[2])/2, (box[1]+box[3])/2
        if score < score_threshold:
            # print(f"----->Ignoring the prediction at x={x_hat} y={y_hat} with score={score}")
            plt.plot(x_hat, y_hat, marker="x", markersize=5, markeredgecolor="white", markerfacecolor="white")
            continue
            
        # print(f"Prediction coordinate x={x_hat} y={y_hat}")
        plt.plot(x_hat, y_hat, marker="x", markersize=5, markeredgecolor="red", markerfacecolor="red")
        plotted_predictions += 1

    plt.title(f"Tube-{tube_idx} Predicted-{plotted_predictions} peaks")
    plt.show()

In [15]:
# frame_to_tensor = A.Compose([ToTensorV2(p=1.0)])

In [16]:
# img = np.tile(frame_data[:, :, None], 3).astype(np.float32)

In [17]:
# b = a(image=img)

In [18]:
# b['image'].shape, type(b['image'])

In [19]:
# for tube in range(ws.shape[0]):
#     frame_data = ws[tube, ...]
#     img = np.tile(frame_data[:,:, None], 3).astype(np.float32)
#     img = frame_to_tensor(image=img)
#     # print(np.tile(frame_data[None,:,:], 3).shape)
#     # img_res = tc.from_numpy(img_res)

In [20]:
# img

In [21]:
# model.eval()

# probe = 0

# for img in range(ws.shape[0]):
#     frame_data = ws[tube, ...]
#     frame = np.tile(frame_data[:,:, None], 3).astype(np.float32)
#     frame = frame_to_tensor(image=frame)
#     with tc.no_grad():
#         prediction = model([frame['image'].to(device)])[0]
#         print(prediction)
#         nms_prediction = apply_nms(prediction, iou_thresh=0.001)
#         plot_tube_detections(frame['image'][0,:,:], nms_prediction, 0.50)
#         probe += 1
#         if probe > 1:
#             break

In [22]:
%%time
model.eval()
probe = 0
predicted_indices_with_score = []

for batch_idx, img_batch in enumerate(data_loader):
    for img_idx, img in enumerate(img_batch):
        tube_idx = batch_idx * data_loader.batch_size + img_idx
        with tc.no_grad():
            prediction = model([img.to(device)])[0]
            nms_prediction = apply_nms(prediction, iou_thresh=0.001)
            for box, score in zip(nms_prediction['boxes'], nms_prediction['scores']):
                tof = (box[0]+box[2])/2
                tube_res = (box[1]+box[3])/2
                predicted_indices_with_score.append([tube_idx, tube_res, tof, score])
            # plot_tube_detections(tube_idx, img[0,:,:], nms_prediction, 0.50)
            probe += 1
            
    # if probe > 1:
    #     break

CPU times: user 37.7 s, sys: 1.01 s, total: 38.8 s
Wall time: 42.9 s


In [23]:
len(predicted_indices_with_score)

130

In [24]:
pred_on_cpu = [[tube, x.cpu(), y.cpu(), z.cpu()] for tube, x, y, z in predicted_indices_with_score]
pred_on_cpu

[[94, tensor(63.9935), tensor(694.4263), tensor(0.0858)],
 [258, tensor(63.4673), tensor(613.6406), tensor(0.9007)],
 [258, tensor(63.5386), tensor(979.6720), tensor(0.8210)],
 [259, tensor(63.0234), tensor(616.9379), tensor(0.9476)],
 [259, tensor(62.8094), tensor(981.2410), tensor(0.8365)],
 [260, tensor(63.7084), tensor(617.9465), tensor(0.8917)],
 [260, tensor(63.9258), tensor(983.8971), tensor(0.8198)],
 [320, tensor(64.2576), tensor(641.8206), tensor(0.1162)],
 [367, tensor(63.4823), tensor(1004.5889), tensor(0.8016)],
 [367, tensor(63.6780), tensor(637.5804), tensor(0.7661)],
 [368, tensor(64.8655), tensor(640.4392), tensor(0.8100)],
 [368, tensor(64.7994), tensor(1005.7531), tensor(0.7299)],
 [369, tensor(63.2543), tensor(1007.6409), tensor(0.8534)],
 [369, tensor(63.0675), tensor(640.8425), tensor(0.8464)],
 [440, tensor(64.0928), tensor(965.4148), tensor(0.7139)],
 [440, tensor(64.1535), tensor(598.5773), tensor(0.4873)],
 [441, tensor(63.0361), tensor(600.3401), tensor(0.837

In [25]:
savepath=f"/mnt/ceph/home/wj1132075/Desktop/CNN_Model_Data/Inferences/RealData/{ws_name}_inferences_TbyT_Ax0Ax1.npz"
np.savez_compressed(savepath, pred_on_cpu)

In [26]:
np.load(savepath)['arr_0']

array([[9.40000000e+01, 6.39934769e+01, 6.94426331e+02, 8.58429000e-02],
       [2.58000000e+02, 6.34672928e+01, 6.13640564e+02, 9.00671363e-01],
       [2.58000000e+02, 6.35386353e+01, 9.79671997e+02, 8.21025431e-01],
       [2.59000000e+02, 6.30233536e+01, 6.16937866e+02, 9.47606087e-01],
       [2.59000000e+02, 6.28094254e+01, 9.81240967e+02, 8.36528897e-01],
       [2.60000000e+02, 6.37083855e+01, 6.17946533e+02, 8.91731679e-01],
       [2.60000000e+02, 6.39257851e+01, 9.83897095e+02, 8.19848061e-01],
       [3.20000000e+02, 6.42576447e+01, 6.41820557e+02, 1.16221152e-01],
       [3.67000000e+02, 6.34823151e+01, 1.00458893e+03, 8.01555693e-01],
       [3.67000000e+02, 6.36779785e+01, 6.37580444e+02, 7.66141713e-01],
       [3.68000000e+02, 6.48654785e+01, 6.40439209e+02, 8.10019970e-01],
       [3.68000000e+02, 6.47993774e+01, 1.00575305e+03, 7.29867995e-01],
       [3.69000000e+02, 6.32543259e+01, 1.00764093e+03, 8.53376567e-01],
       [3.69000000e+02, 6.30675354e+01, 6.40842529e

In [27]:
arr = np.array(predicted_indices_with_score)
arr.shape

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [ ]:
len(arr[arr[:, -1]>=0.5])

### Data parallelism to speedup

In [ ]:
def doInference(img_batch):
    model.eval()
    predicted_indices_with_score = []
    batch_idx, img_batch = img_batch
    print(f"Starting {batch_idx}")
    for img_idx, img in enumerate(img_batch):
        tube_idx = batch_idx * batch_size + img_idx
        with tc.no_grad():
            prediction = model([img.to(device)])[0]
            nms_prediction = apply_nms(prediction, iou_thresh=0.001)
            for box, score in zip(nms_prediction['boxes'], nms_prediction['scores']):
                tof = (box[0]+box[2])/2
                tube_res = (box[1]+box[3])/2
                predicted_indices_with_score.append([tube_idx, tube_res, tof, score])
                # print(f"Inferred batch={batch_idx} tube={tube_idx}")
                
    print(f"Returning results for batch={batch_idx}...........")
    return predicted_indices_with_score

In [ ]:
# %%time
# data_loader = tc.utils.data.DataLoader(ds, batch_size=8, shuffle=False, num_workers=2)
# batch_size = data_loader.batch_size
# pool = mp.Pool(processes=2)

# inferences = pool.map(doInference, enumerate(data_loader))

# pool.close()
# print("pool closed, waiting to join...")
# pool.join()
# print(len(inferences))

In [ ]:
things=[7,5,1,6,4,2]

def worker(thing):
    id, n = thing
    # time.sleep(n)
    print(f"id={id} thing={n} pid={os.getpid()}\n")
    return n
    
pool = mp.Pool(processes=mp.cpu_count())
results = pool.map(worker, enumerate(things))
pool.close()
pool.join()
for r in results:
    print(f"Result {r}") 